In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np
from rdkit import Chem
from rdkit.Chem import PandasTools
import yaml
import pickle
import pandas as pd
from bigbind import *
from run import *

In [3]:
with open("../cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [5]:
activities_unfiltered = pd.read_csv(cfg["bigbind_folder"] + "/activities_unfiltered.csv")

In [24]:
activities = filter_activities(cfg, activities_unfiltered)
smiles2filename = save_all_mol_sdfs(cfg, activities)

Using cached data from filter_activities
Using cached data from save_all_mol_sdfs


In [25]:
activities

,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,protein_accession
0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,420.000000,nM,6.380,P14061
1,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1000.000000,nM,6.000,P14061
2,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,4300.000000,nM,5.370,P14061
3,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1150.000000,nM,5.940,P14061
4,Cc1ccc(S(=O)(=O)N2CCN(Cc3ccccc3)C(=O)[C@@H]2CC...,IC50,=,28000.000000,nM,4.550,P08246
...,...,...,...,...,...,...,...
160451,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)c4ccc...,mixed,=,467.735141,nM,6.330,P34947
160452,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)c4ccc...,mixed,=,30.549211,nM,7.515,P34947
160453,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)Cc4cc...,mixed,=,8.511380,nM,8.070,P34947
160454,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,97.723722,nM,7.010,P13569


In [22]:
filename_col = []
for smiles in activities.canonical_smiles:
    if smiles in smiles2filename:
        filename_col.append(smiles2filename[smiles])
    else:
        filename_col.append("error")

activities["structure_file"] = filename_col

activities = activities.query("structure_file != 'error'").reset_index(drop=True)

In [23]:
activities

,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,protein_accession,structure_file
0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,420.000000,nM,6.380,P14061,chembl_structures/mol_0.sdf
1,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1000.000000,nM,6.000,P14061,chembl_structures/mol_1.sdf
2,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,4300.000000,nM,5.370,P14061,chembl_structures/mol_2.sdf
3,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1150.000000,nM,5.940,P14061,chembl_structures/mol_3.sdf
4,Cc1ccc(S(=O)(=O)N2CCN(Cc3ccccc3)C(=O)[C@@H]2CC...,IC50,=,28000.000000,nM,4.550,P08246,chembl_structures/mol_4.sdf
...,...,...,...,...,...,...,...,...
1007795,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)c4ccc...,mixed,=,467.735141,nM,6.330,P34947,chembl_structures/mol_609763.sdf
1007796,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)c4ccc...,mixed,=,30.549211,nM,7.515,P34947,chembl_structures/mol_539526.sdf
1007797,Cc1[nH]c(/C=C2\C(=O)Nc3ccc(C(=O)N[C@H](C)Cc4cc...,mixed,=,8.511380,nM,8.070,P34947,chembl_structures/mol_539527.sdf
1007798,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,97.723722,nM,7.010,P13569,chembl_structures/mol_609764.sdf
